# Sentiment Analysis Experiments

Goal: predict `sentiment` (positive/neutral/negative) from learner feedback. We escalate from classic ML baselines to advanced transformers and agentic LLM workflows, with explainability, research-ethics checks, and error analysis.

In [ ]:
import pandas as pd
from pathlib import Path

DATA_PATH = Path('data_feedback.xlsx')
df = pd.read_excel(DATA_PATH)
df.head()

## 1. Data audit and ethics guardrails
- **PII scan:** quick heuristic scan to avoid leaking names; expand with policy-based filters for real deployments.
- **Bias check:** review label balance and text length distribution to avoid overfitting to overly positive feedback.
- **Consent note:** ensure data use follows institutional review guidelines before training or sharing models.

In [ ]:
# Quick descriptive stats
label_counts = df['sentiment'].value_counts()
lengths = df['comments'].str.len()
label_counts, lengths.describe()

## 2. Baseline: TF–IDF + Linear Models
Rationale: fast, transparent, and establishes a sanity-check benchmark before using heavier models.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

X_train, X_val, y_train, y_val = train_test_split(df['comments'], df['sentiment'], test_size=0.2, stratify=df['sentiment'], random_state=42)

tfidf_lr = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1,2), min_df=1)), ('clf', LogisticRegression(max_iter=200, class_weight='balanced'))])
tfidf_lr.fit(X_train, y_train)
y_pred = tfidf_lr.predict(X_val)
print(classification_report(y_val, y_pred))

### Explainability for baseline
Use SHAP/LIME to identify influential n-grams driving predictions.

In [ ]:
# Example with LIME (install if needed)
# from lime.lime_text import LimeTextExplainer
# explainer = LimeTextExplainer(class_names=tfidf_lr.classes_)
# exp = explainer.explain_instance(X_val.iloc[0], tfidf_lr.predict_proba, num_features=8)
# exp.show_in_notebook()

## 3. Intermediate: Classical + Character Features
Rationale: character n-grams add robustness to misspellings and colloquial phrases.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer as CharTfidf
char_tfidf_lr = Pipeline([('tfidf', CharTfidf(analyzer='char', ngram_range=(3,5), min_df=1)), ('clf', LogisticRegression(max_iter=200, class_weight='balanced'))])
char_tfidf_lr.fit(X_train, y_train)
print(classification_report(y_val, char_tfidf_lr.predict(X_val)))

## 4. Advanced: Transformer Fine-tuning
Rationale: contextual embeddings capture nuanced sentiment cues.
- **Model:** `distilbert-base-uncased` or similar.
- **Tricks:** class weights, freeze lower layers for small data, early stopping.
- **Evaluation:** macro-F1, calibration curves for threshold tuning.

In [ ]:
# Skeleton; uncomment to run in GPU environment
# from datasets import Dataset
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
# model_name = 'distilbert-base-uncased'
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# dataset = Dataset.from_pandas(df[['comments', 'sentiment']])
# label2id = {lbl:i for i,lbl in enumerate(sorted(df['sentiment'].unique()))}
# id2label = {i:lbl for lbl,i in label2id.items()}
# dataset = dataset.map(lambda x: {'labels': label2id[x['sentiment']]}, remove_columns=['sentiment'])
# dataset = dataset.map(lambda x: tokenizer(x['comments'], truncation=True), batched=True)
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label2id), id2label=id2label, label2id=label2id)
# args = TrainingArguments(output_dir='./sentiment-model', num_train_epochs=6, per_device_train_batch_size=16, per_device_eval_batch_size=32, learning_rate=2e-5, weight_decay=0.01, evaluation_strategy='epoch', load_best_model_at_end=True)
# trainer = Trainer(model=model, args=args, train_dataset=dataset, eval_dataset=dataset, tokenizer=tokenizer)
# trainer.train()

### Transformer explainability
- Integrated gradients or SHAP on transformer outputs to surface token saliency.
- Contrast LIME explanations between the TF–IDF baseline and the transformer.

## 5. Agentic LLM workflow
Rationale: leverage instruction-tuned LLM with chain-of-thought + self-consistency for low-data regimes.
- Prompt template enforces JSON output and short rationale.
- Add uncertainty scoring via disagreement across temperature-sampled responses.
- Guardrails: content filters to avoid unsafe generations and ensure privacy.

In [ ]:
# Pseudocode placeholder for LLM inference
# def llm_sentiment_predict(text):
#     prompt = f"Classify sentiment (positive/neutral/negative) for: {text}. Return JSON with label and short rationale."
#     # call open-source model or API here
#     return {'label': 'positive', 'rationale': 'praise words'}

## 6. Error analysis & ethics review
- Build a **mistake notebook**: list misclassifications with text, predicted vs. gold, and attribution highlights.
- Examine failures by length buckets and by topic (teacher vs. course) to detect bias.
- Document ethical considerations: avoid overclaiming accuracy, respect consent, and minimize harm when providing automated feedback.